# Linear models

In this notebook, we introduce in depth a family of machine learning model called linear models.

## Which part of the model is linear?

As we previously discussed, a predictive model is a mathematical function $f$ that given a matrix of feature $X$ provides some predictions $\hat{y}$. Formally, we have $\hat{y}= f(X)$. Linear models define a certain type of functions $f$:

$$
f{X} = \beta X \\
f{X} = a_0 + a_1 X_1 + a_2 X_2 + \dots + a_n X_n
$$

In other words, a linear model is a function that linearly combined the features of the matrix $X$ to provide a prediction $y$. Let's take a simple regression example with a single feature.

In [ ]:
import pandas as pd

data = pd.read_csv("datasets/penguins_regression.csv")
data.head()

In this dataset, we want to use a linear model that given the flipper length of a penguin, we predict the body mass of the penguin. First, let's have a look at the relationship between these two measurements.

In [ ]:
import matplotlib.pyplot as plt

_, ax = plt.subplots()
ax.scatter(data["Flipper Length (mm)"], data["Body Mass (g)"])
ax.set_xlabel("Flipper length (mm)")
ax.set_ylabel("Body mass (g)")
_ = ax.set_title(
    "Body mass of a penguin in function of its flipper length"
)

We observe that we have a kind of linear relationship: longer is the flipper, heavier is the penguin. A linear model in this context, would be a function that is parametrized to provide a straight line. We can define a function for this purpose.

In [ ]:
def f(x, coef, intercept):
    predictions = coef * x + intercept
    return predictions

This function take as input `x` that corresponds to the flipper length and is then parametrize by `coef` and `intercep`. Those correspond to the $a_0$ and $a_1$ of the equation of the linear model given above.

Let's make a try by defining a value for these two parameter and observe the resulting line on the plot.

In [ ]:
intercept = 0
coef = 20
predicted_body_mass = f(data["Flipper Length (mm)"], coef, intercept)

In [ ]:
_, ax = plt.subplots()
ax.scatter(data["Flipper Length (mm)"], data["Body Mass (g)"])
ax.plot(
    data["Flipper Length (mm)"],
    predicted_body_mass,
    linewidth=3,
    color="tab:orange",
    label="Predictive model",
)
ax.legend()
ax.set_xlabel("Flipper length (mm)")
ax.set_ylabel("Body mass (g)")
_ = ax.set_title(
    "Body mass of a penguin in function of its flipper length"
)

We just built our linear predictive model: passing a `x` value will provide us a body mass prediction.

### Question

- *Given this model, how would you quantified the quality of this predictive model?*
- *Can you provide a set of parameters for which the predictions are more accurate?*

In [ ]:
import numpy as np


def mean_absolute_error(y_true, y_pred):
    return np.mean(np.abs(y_true - y_pred))


mean_absolute_error(data["Body Mass (g)"], predicted_body_mass)

In [ ]:
def mean_squared_error(y_true, y_pred):
    return np.mean((y_true - y_pred) ** 2)


mean_squared_error(data["Body Mass (g)"], predicted_body_mass)

## From manual to automatized model

In the previous exercise, we define some "metric" that define the quality of a predictive model. We could use such a metric to find the optimal predicitive model; the model for which the error is minimum. This metric is also known as **loss function**.

A bit of numerical optimization and applied mathematics tell us that we can try to use gradient descent algorithm and mathematical derivative to find the minimum of the function. But we will just rely on SciPy that implement such algorithm for us. We need to modify a bit the previous error function:

In [ ]:
def f_mean_squared_error(params, data):
    y_pred = f(
        data["Flipper Length (mm)"],
        coef=params[1],
        intercept=params[0],
    )
    return mean_squared_error(data["Body Mass (g)"], y_pred)

In [ ]:
from scipy.optimize import minimize

results = minimize(
    f_mean_squared_error, x0=(intercept, coef), args=data
)
results

So the function `minimize` find a set of parameters with the lowest possible error. We can check the value of the intercept and coefficients.

In [ ]:
results.x

Let's use these parameters to check visually what is the output of such parametric model.

In [ ]:
_, ax = plt.subplots()
ax.scatter(data["Flipper Length (mm)"], data["Body Mass (g)"])
ax.plot(
    data["Flipper Length (mm)"],
    f(data["Flipper Length (mm)"], coef=results.x[1], intercept=results.x[0]),
    linewidth=3,
    color="tab:orange",
    label="Predictive model",
)
ax.legend()
ax.set_xlabel("Flipper length (mm)")
ax.set_ylabel("Body mass (g)")
_ = ax.set_title(
    "Body mass of a penguin in function of its flipper length"
)

We see that this model is much better than our manually defined model. This model is the one minimizing the average of the squared errors.

### Questions

- *Use `sklearn.linear_model.LinearRegression` to `fit` a model.*
- *By looking a the documentation, what are the value of the coefficients of the linear model?*
- *Plot the predictions that you can obtain with `predict` as in the previous plot.*
- *Use `sklearn.metrics.mean_squared_error` to compute the error of this model.*

In [ ]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(data[["Flipper Length (mm)"]], data["Body Mass (g)"])

In [ ]:
model.coef_, model.intercept_

In [ ]:
y_pred = model.predict(data[["Flipper Length (mm)"]])

In [ ]:
from sklearn.metrics import mean_squared_error

mean_squared_error(data["Body Mass (g)"], y_pred)

In [ ]:
_, ax = plt.subplots()
ax.scatter(data["Flipper Length (mm)"], data["Body Mass (g)"])
ax.plot(
    data["Flipper Length (mm)"],
    y_pred,
    linewidth=3,
    color="tab:orange",
    label="Predictive model",
)
ax.legend()
ax.set_xlabel("Flipper length (mm)")
ax.set_ylabel("Body mass (g)")
_ = ax.set_title(
    "Body mass of a penguin in function of its flipper length"
)

## What about other loss functions?

So `LinearRegression` minimized the mean squared error. But we might want to minize the absolute error instead. But first let's check what is the different between the squared error and absolute error.

In [ ]:
xx = np.linspace(-2, 2, num=100)

plt.plot(xx, (xx - 0) ** 2, label="squared error")
plt.plot(xx, np.abs(xx - 0), label="absolute error")
plt.ylabel("Error")
_ = plt.legend()

### Question

- *What is the difference that you can observe between the two type of error?*
- *What is the practical implications?*

Let's go in a situation that a scientist made measurements of penguins but "dg" instead of "g". We will create a new dataframe containing the new measurements.

In [ ]:
n_fake_penguins = 50
fake_flipper_length = np.random.uniform(low=220, high=230, size=n_fake_penguins)
fake_body_mass = np.random.uniform(low=550, high=650, size=n_fake_penguins)

In [ ]:
new_data = pd.concat(
    [
        data,
        pd.DataFrame({
            "Flipper Length (mm)": fake_flipper_length,
            "Body Mass (g)": fake_body_mass
        })
    ]
)

We can quickly plot the dataset to have an idea on the impact of the error commited by the scientist.

In [ ]:
_ = new_data.plot.scatter(x="Flipper Length (mm)", y="Body Mass (g)")

So we observe those new data samples that could be consider as "ouliers".

### Questions

- *Fit a `LinearRegression` model on this new dataset.*
- *Are the coefficients significantly different from the previous model?*
- *Display the predictions of the model and compare it with our previous model. Do you observe any difference?*

In [ ]:
model = LinearRegression()
model.fit(new_data[["Flipper Length (mm)"]], new_data["Body Mass (g)"])
y_pred = model.predict(new_data[["Flipper Length (mm)"]])

In [ ]:
model.coef_, model.intercept_

In [ ]:
_, ax = plt.subplots()
ax.scatter(new_data["Flipper Length (mm)"], new_data["Body Mass (g)"])
ax.plot(
    new_data["Flipper Length (mm)"],
    y_pred,
    linewidth=3,
    color="tab:orange",
    label="Predictive model",
)
ax.legend()
ax.set_xlabel("Flipper length (mm)")
ax.set_ylabel("Body mass (g)")
_ = ax.set_title(
    "Body mass of a penguin in function of its flipper length"
)

### Questions

- *Repeat the previous experiment and fit a `sklearn.linear_model.QuantileRegressor`.*

In [ ]:
from sklearn.linear_model import QuantileRegressor

model = QuantileRegressor(solver="highs")
model.fit(new_data[["Flipper Length (mm)"]], new_data["Body Mass (g)"])
y_pred = model.predict(new_data[["Flipper Length (mm)"]])

In [ ]:
_, ax = plt.subplots()
ax.scatter(new_data["Flipper Length (mm)"], new_data["Body Mass (g)"])
ax.plot(
    new_data["Flipper Length (mm)"],
    y_pred,
    linewidth=3,
    color="tab:orange",
    label="Predictive model",
)
ax.legend()
ax.set_xlabel("Flipper length (mm)")
ax.set_ylabel("Body mass (g)")
_ = ax.set_title(
    "Body mass of a penguin in function of its flipper length"
)

### Questions

- *Create two new models where you should get an estimator for the 10th and 90th quantiles instead of the default median. You can check the parameter `quantile` of the `QuantileRegressor`.*
- *Plot the the predictions that you get with the 10th, 50th, and 90th quantiles. What the 10-90 quantiles coverage represent?*

In [ ]:
model = QuantileRegressor(solver="highs", quantile=0.1)
model.fit(
    new_data[["Flipper Length (mm)"]], new_data["Body Mass (g)"]
)
y_pred_10 = model.predict(new_data[["Flipper Length (mm)"]])

In [ ]:
model = QuantileRegressor(solver="highs", quantile=0.9)
model.fit(
    new_data[["Flipper Length (mm)"]], new_data["Body Mass (g)"]
)
y_pred_90 = model.predict(new_data[["Flipper Length (mm)"]])

In [ ]:
_, ax = plt.subplots()
ax.scatter(new_data["Flipper Length (mm)"], new_data["Body Mass (g)"])
ax.plot(
    new_data["Flipper Length (mm)"],
    y_pred,
    linewidth=3,
    color="tab:orange",
    label="quantile=0.5",
)
ax.plot(
    new_data["Flipper Length (mm)"],
    y_pred_10,
    linewidth=3,
    color="tab:green",
    label="quantile=0.1",
)
ax.plot(
    new_data["Flipper Length (mm)"],
    y_pred_90,
    linewidth=3,
    color="tab:red",
    label="quantile=0.9",
)
ax.legend()
ax.set_xlabel("Flipper length (mm)")
ax.set_ylabel("Body mass (g)")
_ = ax.set_title(
    "Body mass of a penguin in function of its flipper length"
)

## What about multiple features in `X`?

Up to now, we saw a use case where we dealt with a single feature and a single target. The previous experiment can be extended to multiple features. Let's create a (fake) feature called "Flipper Width".

In [ ]:
data["Flipper Width (mm)"] = data["Flipper Length (mm)"] / 10 + np.random.randn(len(data))
data

It is a bit more complex but we can still plot the data in a 3-dimensional scatter plot

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
ax.scatter(
    data["Flipper Length (mm)"],
    data["Flipper Width (mm)"],
    data["Body Mass (g)"],
)

We will use `LinearRegression` and use the 2 flipper measurements as input data and predict the body mass as previously done.

In [ ]:
model = LinearRegression()
model.fit(
    data[["Flipper Length (mm)", "Flipper Width (mm)"]],
    data["Body Mass (g)"],
)

We can check the coefficients and intercept to check the difference compare to our previous example.

In [ ]:
model.coef_, model.intercept_

We observe that we have an additional coefficient corresponding to a weight associated with the new "Flipper Width". We can now plot the predictions obtained with the model.

In [ ]:
xx, yy = np.meshgrid(
    data["Flipper Length (mm)"], data["Flipper Width (mm)"]
)

In [ ]:
y_pred = model.predict(np.vstack([xx.ravel(), yy.ravel()]).T).reshape(xx.shape)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
ax.scatter(
    data["Flipper Length (mm)"],
    data["Flipper Width (mm)"],
    data["Body Mass (g)"],
)
_ = ax.plot_surface(xx, yy, y_pred)

In this case, we observe that the straight line in the 1-D case become a plane in the 2-D case.

## Feature engineering or getting a non-linear decision function

Up to now, we saw that the predictions boil down to a linear combination of the input feature. Sometimes, this could be limiting when there is a non-linear relationship between the input features and the target. Let's generate some synthetic data where this is the case.

In [ ]:
n_samples = 200
X = np.linspace(-5, 5, n_samples)
y = X + 2 * np.cos(2 * np.pi * X) + 2 * np.random.rand(n_samples)
X = X.reshape(-1, 1)

In [ ]:
_ = plt.scatter(X, y)

We generated true target as a combination of a linear trend with and additional periodic signal and a bit of noise.

### Questions

- *Fit a `LinearRegression` model.*
- *Show the predictions obtained with this model.*
- *Add a new column in `X` that corresponds to `np.cos(2 * np.pi * X[:, 0]`*
- *Fit again a `LinearRegression` model and display the prediction. Is the decision function linear?*

In [ ]:
model = LinearRegression().fit(X, y)

In [ ]:
plt.scatter(X, y)
plt.plot(X, model.predict(X), linewidth=3, color="tab:orange")

In [ ]:
X = np.concatenate(
    [X, np.cos(2 * np.pi* X)], axis=1
)

In [ ]:
model = LinearRegression().fit(X, y)
plt.scatter(X[:, 0], y)
plt.plot(X[:, 0], model.predict(X), linewidth=3, color="tab:orange")

In [ ]:
model.coef_

## Feature engineering, overfitting, and regularization

We might not know in advance what the period of the signal. So instead, we could add several columns in `X` and vary the period.

In [ ]:
n_samples = 200
X = np.linspace(-5, 5, n_samples)
y = X + 2 * np.cos(2 * np.pi * X) + 2 * np.random.rand(n_samples)
X = X.reshape(-1, 1)

n_periods = 10
for i in range(1, n_periods):
    X = np.concatenate(
        [X, np.cos(i * np.pi* X[:, [0]])], axis=1
    )

In [ ]:
model = LinearRegression().fit(X, y)
plt.scatter(X[:, 0], y)
plt.plot(X[:, 0], model.predict(X), linewidth=3, color="tab:orange")

In [ ]:
model.coef_

We see that in this case, our model pick up the right trend.

### Questions

- *Repeat the same experiment by increasing the number of columns, let say 1,000 to have a large range of cosine periods.*
- *What do you observe?*
- *Instead of a `LinearRegression`, fit a `sklearn.linear_model.Ridge` model with the default parameters.*

In [ ]:
from sklearn.linear_model import Ridge

model = Ridge().fit(X, y)
plt.scatter(X[:, 0], y)
plt.plot(X[:, 0], model.predict(X), linewidth=3, color="tab:orange")

### Questions

- *Vary the `alpha` parameter (from 1e-6 to 1e3) of the `Ridge` model.*
- *Check the predictions that you obtain. What is the role of the `alpha` parmameter.*

In [ ]:
model = Ridge(alpha=1e3).fit(X, y)
plt.scatter(X[:, 0], y)
plt.plot(X[:, 0], model.predict(X), linewidth=3, color="tab:orange")

In [ ]:
model = Ridge(alpha=1e-6).fit(X, y)
plt.scatter(X[:, 0], y)
plt.plot(X[:, 0], model.predict(X), linewidth=3, color="tab:orange")

### Questions?

- *Repeat the same experiment using a `sklearn.linear_model.Lasso` model. You can have a look at the `coef_` attribute as well to get more insights.*

In [ ]:
from sklearn.linear_model import Lasso

model = Lasso(alpha=1e-1).fit(X, y)
plt.scatter(X[:, 0], y)
plt.plot(X[:, 0], model.predict(X), linewidth=3, color="tab:orange")

In [ ]:
model.coef_

## From regression to classification problem

Up to now, we focus on a regression problem. However, the above technique is not intended to be used for classification. Let's use a classification dataset to see the difference with the regression that we observe above.

In [ ]:
import pandas as pd

data = pd.read_csv("datasets/penguins_classification.csv")
X = data[["Culmen Length (mm)"]]
y = (data["Species"] == "Adelie").astype(int)

In [ ]:
plt.scatter(X, y)
plt.ylabel("Is Adelie penguin?")
_ = plt.xlabel("Culmen Length (mm)")

We see that the target takes only the value 0 or 1. Indeed, we could use the sigmoid function in order to have constraint the value of any real number to be in the range [0, 1]. This is called a `LogisticRegression`.

In [ ]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression().fit(X, y)
y_pred = model.predict(X)

In [ ]:
plt.scatter(X, y, label="truth")
plt.scatter(X, y_pred, label="predictions")
plt.ylabel("Is Adelie penguin?")
plt.xlabel("Culmen Length (mm)")
_ = plt.legend()

So it means that in practice, the classification give "hard" prediction that should be 0 or 1). However, the sigmoid function can provide continuous values that relates to the probability to be the class 1.

In [ ]:
y_proba = model.predict_proba(X)
y_proba[:5]

In [ ]:
plt.scatter(X, y, label="truth")
plt.scatter(X, y_proba[:, 1], label="predictions")
plt.ylabel("Is Adelie penguin?")
plt.xlabel("Culmen Length (mm)")
_ = plt.legend()

## The importance of preprocessing

For the moment, we use only a regression or classification model without doing any preprocessing on the data. However, it could be an issue. Let's dive into this.

### Questions

- *Load the `sklearn.datasets.load_iris` datasets.*
- *Fit a `LogisticRegression` model.*
- *Do you observe any issue?*

In [ ]:
from sklearn.datasets import load_iris

X, y = load_iris(return_X_y=True)
model = LogisticRegression().fit(X, y)

In [ ]:
model.n_iter_

### Questions

- *Increase the number of iterations `max_iter` and check if it solves the problem.*
- *Is there alternative solution that could be better?*

In [ ]:
model = LogisticRegression(max_iter=1_000).fit(X, y)

In [ ]:
model.n_iter_

In [ ]:
X.mean(axis=0)

In [ ]:
X.std(axis=0)

In [ ]:
X_normalize = (X - X.mean(axis=0)) / X.std(axis=0)

In [ ]:
model.fit(X_normalize, y)

In [ ]:
model.n_iter_

### Questions

- *Instead of the manual scaling above, use the `sklearn.preprocessing.StandardScaler` model.*
- *Call `fit` and `transform` methods on the data to see the effect.*

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler().fit(X)

In [ ]:
scaler.mean_

In [ ]:
scaler.scale_

In [ ]:
scaler.transform(X)[:5]

Now, let's imagine that you have a training and testing set.

### Questions

- *On which dataset should I call `fit`?*
- *On which dataset should I call `transform`?*

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=0
)

In [ ]:
scaler.fit(X_train)
X_train_normalize = scaler.transform(X_train)

In [ ]:
X_test_normalize = scaler.transform(X_test)

In [ ]:
model.fit(X_train_normalize, y_train)

In [ ]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test, model.predict(X_test_normalize))

## The scikit-learn `Pipeline`


To simplify the processing and to not make mistake regarding when calling `fit` and `transform`, one can use `Pipeline`. A pipeline is just a chain of steps that would transform the data with a final steps that is usually a regressor or a classifier. The way to use the pipeline is identical of using the last steps.

In [ ]:
from sklearn.pipeline import Pipeline

model = Pipeline(
    steps=[
        ("scaler", StandardScaler()),
        ("linear_model", LogisticRegression()),
    ]
)
model

When fitting the model, internally we call `fit` and `transform` of the `StandardScaler` and the `fit` method of the `LogsiticRegression`.

In [ ]:
model.fit(X_train, y_train)

We can access a pipeline step using the Python indexing. For instance, we can get the coefficient of the `LogisticRegression`.

In [ ]:
model[-1].coef_

When calling `predict`, it will be equivalent to call `transform` of the `StandardScaler` followed by the `predict` method of the `LogisticRegression`.

In [ ]:
y_pred = model.predict(X_test)
accuracy_score(y_test, y_pred)

## Dealing with categorical data

Up to now, we used data that were only numerical values. This is quite restrictive and many datasets come with some type of data that are called categorical data. We can recognize a categorical feature because the feature values are limited to a small cardinality of choices (e.g. countries, sex, etc.).

There are two questions to have in mind when it comes to deal with these specific type of data:

- How to handle to non-numeric data?
- What is the impact of the type of encoding on the underlying model?

Let's first have a try on a more real dataset that will contain heterogeneous types (numerical and categorical).

In [ ]:
data = pd.read_csv("datasets/adult-census.csv")

In [ ]:
X = data.drop(columns=["class"])
y = data["class"]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=0
)

In [ ]:
X_train.head()

### Questions

- *Fit a `LogisticRegression` model on the training set.*
- *What do you get?*

In [ ]:
model = LogisticRegression().fit(X_train, y_train)

In [ ]:
X.info()

### Encoding categories with a specific order

The simplest way that one can think of is to replace a category by a numerical value. This is indeed the job of the `sklearn.preprocessing.OrdinalEncoder`. Let's give an example of the output such model.

In [ ]:
categorical_features = [
    "workclass",
    "education",
    "marital-status",
    "occupation",
    "relationship",
    "race",
    "sex",
    "native-country",
]

In [ ]:
X[categorical_features]

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

encoder = OrdinalEncoder().fit(X[categorical_features])
encoder.categories_

In [ ]:
encoder.transform(X[categorical_features])

### Question

- *What is the impact of using such encoding on the modelling used by a linear model?*

### One-hot encoding

Another solution and more desired with linear model is to create a matrix of zeros, with the number of columns corresponding to a category. When the feature value corresponds to a category, then the entry is labelled with a one (and so the name one-hot).

In [ ]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(sparse_output=False)
encoder.fit(X[categorical_features])
encoder.transform(X[categorical_features])

## Dealing with heterogeneous data in a single predictive pipeline

Now that we know how to deal with numerical and categorical data, we show how to use `ColumnTransformer` that is a transformer allowing to apply a specific transformer (or pipeline of transformers) to a list of columns.

The `ColumnTransformer` should be think as a 3-steps procedure: split columns by group (provided by the user), transform each individual group of columns, and combine the resulting transformed data.

In [ ]:
from sklearn.compose import ColumnTransformer

numerical_column = [
    "age", "capital-gain", "capital-loss", "hours-per-week"
]
preprocessor = ColumnTransformer(
    transformers=[
        ("cat_preprocessor", OneHotEncoder(), categorical_features),
        ("num_preprocessor", StandardScaler(), numerical_column)
    ]
)
model = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("linear_model", LogisticRegression(max_iter=1_000)),
])

In [ ]:
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)
accuracy_score(y_test, y_pred)

In [ ]:
model[:-1].transform(X_train).A